In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pylab as plt

# This notebook matches the OCC-ACS codes crosswalks on the BLS codes and OMN codes actually used

In [ ]:
# paths
data_local = "../data/"
data_out = '../results/data_out/'

Read in data: 1) occupations in BLS, 2) occupations in OMN, 3) crosswalk

In [ ]:
# occuaptions to sector data 2018
filename_occ_sect = "BLS/oesm18nat/national_M2018_dl.xlsx"
tab = "national_dl"
df_occ_sect_2018 = pd.read_excel(data_local + filename_occ_sect, sheet_name = tab, header = 0)
df_occ_2018 = df_occ_sect_2018[['OCC_CODE', 'OCC_TITLE', 'OCC_GROUP']].drop_duplicates().sort_values('OCC_CODE')

In [ ]:
df_occ_2018['OCC_GROUP'].hist(xlabelsize=20, ylabelsize=23)

In [ ]:
# occupations in occupational mobility network (OMN) with ASEC
filename_omn_meta = "occ_names_employment_asec.csv"
df_IPUMS = pd.read_csv(data_local + filename_omn_meta, header=0)

In [ ]:
df_IPUMS

In [ ]:
# labour occupations crosswalk
filename_asec_soc = "occ_names_class_asec_soc_map.csv"
df_asec_soc = pd.read_csv(data_local + filename_asec_soc, header=0, index_col=0)

In [ ]:
df_asec_soc

# We will check how well the data from the IPUMS and BLS map on the crosswalk. For IPUMS

In [ ]:
a = df_asec_soc.merge(df_IPUMS, on='Code', how='outer', suffixes=('', '_a'))

In [ ]:
a['Code'].isnull().sum()

In [ ]:
a['soc_code'].isnull().sum()

In [ ]:
a[a['soc_code'].isnull()][['Label_a', 'EMP_2019', 'EMP_2010_2019_avg']]

In the crosswalk there are 23 asec codes that do not have a corresponding soc code

# For the BLS we find:

We first note that many codes merge on the detailed level when the broad level has the same name and potentially more reliable information in the BLS dataset. We rectify this as follows

In [ ]:
# dataframe b matches the BLS dataset on the OCC-ASEC crosswalk
b = df_occ_2018
b = b.merge(df_asec_soc, how='outer', right_on='soc_code', left_on="OCC_CODE")

In [ ]:
b[~b['Code'].isnull()]['OCC_GROUP'].hist(xlabelsize=23, ylabelsize=23)

In [ ]:
b.iloc[104:110]

In [ ]:
# If broad and detailed occupation titles are the same, we assign the crosswalk to the 'broad' level
def assign_broad(group, title):
    if ((group == 'detailed') and ((b['OCC_TITLE'] == title).sum() > 1)):
        loc = b.index[(b['OCC_TITLE'] == title) & (b['OCC_GROUP'] == 'broad')][0]
        return b['OCC_CODE'].loc[loc]
    
b['code_new'] = b.apply(lambda x: assign_broad(x['OCC_GROUP'], x['OCC_TITLE']), axis=1)
df_det_to_broad = b[['code_new', 'soc_code']].rename(columns={'code': 'code_old'}).dropna()

Looking at the number of non-matches we see:

In [ ]:
# unused BLS codes
b['soc_code'].isnull().sum()

In [ ]:
# unused crosswalk codes
b['OCC_CODE'].isnull().sum()

The BLS data contains codes from all levels of detail. We only need one of those levels to match. This explains the high number of non-matches (966). However, the other way around should result in zero (0) non-matches, but is 27. We see that those mismatches are mostly due to aggregation codes ending on X, XXX, YY, or similar, as well as a few military codes

In [ ]:
# the crosswalk has XX codes, which BLS does not have. We fit these manually
print(b[b['OCC_CODE'].isnull()]['soc_code'].to_list())

In [ ]:
print(b[b['OCC_CODE'].isnull()]['Code'].to_list())

The above codes we fit manually on the BLS data, by using information on their original names and description of change. We use the following csv file to help with the manual fitting

In [ ]:
pd.DataFrame(b[b['OCC_CODE'].isnull()]).to_csv(data_out + 'mismatched_codes_asec_soc.csv')

In [ ]:
# manually fitted result:
df_aggregate_codes = pd.read_csv(data_local + "manual_matched_asec_soc.csv", dtype=object)
df_aggregate_codes.head()

In [ ]:
df_manual_codes = pd.read_csv(data_local + "manual_insertion_asec_soc.csv", dtype=object, usecols=[0,1])
df_manual_codes.head()

In [ ]:
df_asec_soc_imputed = df_asec_soc.astype(str).merge(df_aggregate_codes, on='Code', how='outer')
df_asec_soc_imputed = df_asec_soc_imputed.astype(str).merge(df_manual_codes, on='Code', how='outer', suffixes=('_x', ''))
df_asec_soc_imputed.replace('nan', np.NaN, inplace=True)
df_asec_soc_imputed.code_impute.fillna(df_asec_soc_imputed.code_impute_x, inplace=True)
df_asec_soc_imputed.code_impute.fillna(df_asec_soc_imputed.soc_code, inplace=True)
df_asec_soc_imputed.replace(list(df_det_to_broad.soc_code), list(df_det_to_broad.code_new), inplace=True)

In [ ]:
df_asec_soc_imputed.drop(['soc_code', 'code_impute_x'], axis=1, inplace=True)
df_asec_soc_imputed.drop_duplicates(inplace=True)

Using these new codes, as well as the above detailed-to-broad list we try again:

In [ ]:
b = df_occ_2018
b = b.merge(df_asec_soc_imputed, how='outer', left_on='OCC_CODE', right_on='code_impute')

In [ ]:
b['Code'].isnull().sum()

In [ ]:
b['OCC_CODE'].isnull().sum()

All we have left now are fising and hunting workers, and military occupations, which we both drop

In [ ]:
b[b['OCC_CODE'].isnull()]

In [ ]:
b[~b['Code'].isnull()]['OCC_GROUP'].dropna().count()

Most codes now match on broad categories

In [ ]:
b[~b['Code'].isnull()]['OCC_GROUP'].hist(xlabelsize=23, ylabelsize=23)

We can now thus define a prep data analysis crosswalk as follows

In [ ]:
df_BLS_2018_prep = b[['OCC_CODE', 'OCC_TITLE', 'Code']].dropna().drop_duplicates()

In [ ]:
# All manual codes added
set(df_aggregate_codes.code_impute) - set(df_BLS_2018_prep.OCC_CODE)

In [ ]:
# all manual codes added
set(df_manual_codes.code_impute) - set(df_BLS_2018_prep.OCC_CODE)

In [ ]:
# all original codes included except XX codes, and 23-1010 (changed to 23-1000), 
# and 45-3010 (fishing and hunting workers) which is not in the national BLS data
set([str(x)[:6] for x in df_asec_soc.soc_code]) - set([x[:6] for x in df_BLS_2018_prep.OCC_CODE])

In [ ]:
df_BLS_2018_prep.to_csv(data_out + 'SOC_to_IPUMS.csv')

# Next to do is to merge the two to get a combined crosswalk to get IPUMS to BLS data

We rescale any many-to-one relations with their total employment

In [ ]:
BLS_IPUMS_crosswalk = df_BLS_2018_prep.set_index('OCC_CODE')

BLS_IPUMS_crosswalk = pd.get_dummies(BLS_IPUMS_crosswalk.Code)
BLS_IPUMS_crosswalk = BLS_IPUMS_crosswalk.groupby(BLS_IPUMS_crosswalk.index).sum()
BLS_IPUMS_crosswalk = BLS_IPUMS_crosswalk.T.groupby(BLS_IPUMS_crosswalk.T.index).sum().T

In [ ]:
# There are no occupations without a counterpart
print(len(BLS_IPUMS_crosswalk[BLS_IPUMS_crosswalk.sum(axis=1) == 0].head()))
print(len(BLS_IPUMS_crosswalk.T[BLS_IPUMS_crosswalk.sum() == 0].head()))

In [ ]:
# 54 IPUMS occupations match on more than 1 BLS occupation
# Vice versa, 2 BLS occupations match on multiple IPUMS occupations
# Our main crosswalk is from BLS to ipums, so the 52 many-to-one are not a problem, but
# the 2 one-to-many are. We deal with them manually
print(len(BLS_IPUMS_crosswalk[BLS_IPUMS_crosswalk.sum(axis=1) >1]))
print(len(BLS_IPUMS_crosswalk.T[BLS_IPUMS_crosswalk.sum() >1]))

In [ ]:
BLS_IPUMS_crosswalk[BLS_IPUMS_crosswalk.sum(axis=1) >1]

In [ ]:
# This helper function prints the names of all occupations that match, and their employment totals
def wm(row):
    s = 0
    for col in row.index:
        if row[col] > 0:
            print(int(col))
            val = df_IPUMS[df_IPUMS['Code'] == int(col)]['EMP_2010_2019_avg'].values[0]
            print(val)
            s += val
    print(s)

In [ ]:
# we print 
BLS_IPUMS_crosswalk.loc[BLS_IPUMS_crosswalk.sum(axis=1) >1].apply(lambda r: wm(r), axis=1)

In [ ]:
# only 2 items, so set manually the fractions of total employment:
BLS_IPUMS_crosswalk.loc['13-1020', '510'] = 15442.0234 / 541587.255
BLS_IPUMS_crosswalk.loc['13-1020', '520'] = 226654.904 / 541587.255
BLS_IPUMS_crosswalk.loc['13-1020', '530'] = 299490.327 / 541587.255

BLS_IPUMS_crosswalk.loc['51-4022', '7920'] = 13292.424 / 29839.588
BLS_IPUMS_crosswalk.loc['51-4022', '7940'] = 16547.164 / 29839.588

Now SOC <--> IPUMS has some n-1 relations, but not vice versa. So SOC --> IPUMS is fine to use, but IPUMS --> SOC needs more care

In [ ]:
BLS_IPUMS_crosswalk.to_csv(data_out + '2018_BLS_IPUMS_crosswalk_proportional.csv')

In [ ]:
# node lists:
ipums_node_list = df_IPUMS.copy()
ipums_node_list.set_index('Code', inplace=True)
ipums_node_list.index = ipums_node_list.index.astype(str)
ipums_node_list = ipums_node_list.loc[BLS_IPUMS_crosswalk.columns]
ipums_node_list.to_csv(data_out + '2018_BLS_IPUMS_crosswalk_nodelist_IPUMS.csv')

bls_nodelist = pd.read_csv(data_out + 'occ_names_bls_minor_major.csv', index_col=0)
bls_nodelist.set_index('OCC_CODE', inplace=True)
#bls_nodelist = bls_nodelist.loc[BLS_IPUMS_crosswalk.index]
bls_nodelist.to_csv(data_out + '2018_BLS_IPUMS_crosswalk_nodelist_BLS.csv')

# Crosswalk other way around: IPUMS to BLS

In [ ]:
df_BLS_2018_prep

In [ ]:
IPUMS_BLS_crosswalk = df_BLS_2018_prep.set_index('Code')

IPUMS_BLS_crosswalk = pd.get_dummies(IPUMS_BLS_crosswalk.OCC_CODE)
IPUMS_BLS_crosswalk = IPUMS_BLS_crosswalk.groupby(IPUMS_BLS_crosswalk.index).sum()
IPUMS_BLS_crosswalk = IPUMS_BLS_crosswalk.T.groupby(IPUMS_BLS_crosswalk.T.index).sum().T

In [ ]:
# There are no occupations without a counterpart
print(len(IPUMS_BLS_crosswalk[IPUMS_BLS_crosswalk.sum(axis=1) == 0].head()))
print(len(IPUMS_BLS_crosswalk.T[IPUMS_BLS_crosswalk.sum() == 0].head()))

In [ ]:
# 54 IPUMS occupations match on more than 1 BLS occupation
# Vice versa, 2 BLS occupations match on multiple IPUMS occupations
# Our main crosswalk is from BLS to ipums, so the 52 many-to-one are not a problem, but
# the 2 one-to-many are. We deal with them manually
print(len(IPUMS_BLS_crosswalk[IPUMS_BLS_crosswalk.sum(axis=1) >1]))
print(len(IPUMS_BLS_crosswalk.T[IPUMS_BLS_crosswalk.sum() >1]))

In [ ]:
bls_emp = pd.read_csv(data_out + 'occ_names_bls_minor_major.csv', index_col=0)[['OCC_CODE', 'TOT_EMP']]

In [ ]:
bls_emp

In [ ]:
# This helper function prints the names of all occupations that match, and their employment totals
def wm(row):
    print(row.name)
    s = 0
    cols = []
    emps = []
    for col in row.index:
        if row[col] > 0:
            cols += [col]
            val = bls_emp[bls_emp['OCC_CODE'] == col]['TOT_EMP'].values[0]
            emps += [val]
            s += val
    print(cols)
    print(emps)
    # set new values
    for col, emp in list(zip(cols, emps)):
        IPUMS_BLS_crosswalk.loc[row.name, col] = emp / s
    print(s)

In [ ]:
# we print 
IPUMS_BLS_crosswalk.loc[IPUMS_BLS_crosswalk.sum(axis=1) >1].apply(lambda r: wm(r), axis=1)

In [ ]:
IPUMS_BLS_crosswalk.to_csv(data_out + '2018_IPUMS_BLS_crosswalk_proportional.csv')